In [2]:
import os
from tqdm import tqdm
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from PIL import Image

In [3]:
# ---- Configurations ---- #
base_model_id = "runwayml/stable-diffusion-v1-5"  # or your custom fine-tuned base model
lora_path = "./lora_output"                      # folder containing your LoRA weights
textual_inversion_path = "./textual_inversion_output"    # folder containing your .bin or .pt or .safetensors file

output_dir = "data/LoRA/"

os.makedirs(output_dir, exist_ok=True)

# Number of images per class to generate
num_images_per_class = 300

# Prompts for each class
class_prompts = {
    "benign": f"an ultrasound image showing a benign breast lesion with smooth borders",
    "malignant": f"an ultrasound image of a malignant breast lesion with irregular borders",
    "normal": f"an ultrasound image showing normal breast tissue without any tumor or lesion"
}

device = "cuda" if torch.cuda.is_available() else "cpu"


In [4]:
# ----------------- LOAD PIPELINE ----------------- #
pipe = StableDiffusionPipeline.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    safety_checker=None,
    scheduler=DPMSolverMultistepScheduler.from_pretrained(base_model_id, subfolder="scheduler")
).to(device)

# ---- Load Textual Inversion ---- #
# pipe.load_textual_inversion(textual_inversion_path, token=custom_token)
# ---- Load LoRA weights ---- #
pipe.unet.load_attn_procs(lora_path)
# Optional: Enable memory-efficient attention if you're on a limited GPU
pipe.enable_xformers_memory_efficient_attention()

Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 13.83it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
/home/fa926284/Documents/Repos/BUSI_diffusion/venv/lib/python3.10/site-packages/diffusers/loaders/unet.py:212: FutureWarning: `load_attn_procs` is deprecated and will be removed in version 0.40.0. Using the `load_attn_procs()` method has been deprecated and will be remo

In [5]:
# ----------------- AUGMENTATION LOOP ----------------- #
os.makedirs(output_dir, exist_ok=True)

for cls, prompt in class_prompts.items():
    save_path = os.path.join(output_dir, cls)
    os.makedirs(save_path, exist_ok=True)

    print(f"Generating {num_images_per_class} images for class: {cls}")
    for i in tqdm(range(num_images_per_class)):
        seed = 1000 + i
        generator = torch.Generator(device=device).manual_seed(seed)
        image = pipe(prompt, num_inference_steps=50, generator=generator).images[0]

        img_path = os.path.join(save_path, f"lora_{cls}_{i:03d}.png")
        image.save(img_path)

print(f"\n✅ Augmentation complete. Images saved to {output_dir}/")


Generating 300 images for class: benign


100%|██████████| 300/300 [19:22<00:00,  3.88s/it]


Generating 300 images for class: malignant


100%|██████████| 300/300 [19:46<00:00,  3.95s/it]


Generating 300 images for class: normal


100%|██████████| 300/300 [19:29<00:00,  3.90s/it]


✅ Augmentation complete. Images saved to data/LoRA//
